## Prepare the data

### Setup the tokenizer

In [5]:
from transformers import AutoTokenizer

# Ensure the order
SPECIAL_TOKENS = ["<persuader>", "<persuadee>"]
MPATH = "nasheed/rl-grp-prj-gpt2-baseagent"

tokenizer = AutoTokenizer.from_pretrained(MPATH)
tokenizer.add_special_tokens({'additional_special_tokens': SPECIAL_TOKENS})
tokenizer.pad_token = tokenizer.eos_token

### Setup the data modules

In [6]:
from datautils import DialogDataModule, DialogBatcher


DPATH = r"..\data\persuasionforgood_corpus"

is_pursuader = True
batcher = DialogBatcher(tokenizer)
dm = DialogDataModule(
    DPATH,
    batcher=batcher,
    batch_size=1,
    split_data=False,
    purpose_text="Convince people to donate to charity.",
    is_pursuader=is_pursuader
)

# Prepare the model

In [3]:
from modelling import DialogAgent


is_pursuader = True
ROLE = "persuader" if is_pursuader else "persuadee"
SAVE_PATH = ""
CKPT = f"../models/base/persuader/baseagent-persuader-epoch=2-val_loss=2.44.ckpt"

model = DialogAgent.load_from_checkpoint(CKPT)
print("Loaded checkpoint")

Loaded checkpoint


## Manual Trials

In [8]:
import torch

purpose_text = "Convince people to donate to charity. <persuader>Hi <persuadee>"

for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(purpose_text + input(">> User") + "<persuader>", return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.clm.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print("GPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

DialoGPT: I'm doing good. 
DialoGPT: I'm fine. 
DialoGPT: Save the Children
DialoGPT: Save the Children is an international non-governmental organization that promotes children's rights, provides relief and helps support children in developing countries.
DialoGPT: Because you are a good person. 
